In [1]:
from textblob import TextBlob
import requests
import pandas as pd
import json
import pandas as pd
import matplotlib.pyplot as plt
import requests
import pandas as pd
import json
import datetime as dt
import numpy as np
from scipy.stats import ks_2samp
import numpy as np
import matplotlib.patches as patches
import seaborn as sns
from pytrends.request import TrendReq
import pandas as pd
from textblob import TextBlob
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax
import matplotlib.dates as mdates
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.multitest import multipletests

import torch
from collections import Counter
import re
from scipy import stats
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu
import matplotlib.ticker as ticker
def add_weekday_month_names(df):
    # Ensure 'created_at' is a datetime column
    df['created_at'] = pd.to_datetime(df['created_at'])

    # Extract the weekday and month
    df['weekday'] = df['created_at'].dt.weekday
    df['month'] = df['created_at'].dt.month

    # Weekday and month name mappings
    weekday_names = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 
                     4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
    month_names = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 
                   5: 'May', 6: 'June', 7: 'July', 8: 'August', 
                   9: 'September', 10: 'October', 11: 'November', 12: 'December'}

    # Convert weekday and month numbers to names
    df['weekday'] = df['weekday'].map(weekday_names)
    df['month'] = df['month'].map(month_names)

    return df

import matplotlib as mpl

mpl.rcParams['axes.labelsize'] = 16
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['legend.fontsize'] = 16
import matplotlib.patches as mpatches

In [2]:
analysis_start_date = '2013-01-01'
analysis_end_date = '2015-04-30'


In [3]:
df_emot = pd.read_csv('packages_emotions.csv')
df_sentiment= pd.read_csv('packages_sentiment.csv',index_col=[0])
df_topic= pd.read_csv('packages_topics.csv',index_col=[0])

In [4]:
df_emot = df_emot[['created_at','test_id','cleaned_text','impressions','clicks','ctr','emotion','confidence']]
df_emot = df_emot.rename(columns={'confidence': 'emotion_score'})
df_topic = df_topic.rename(columns={'labels': 'topic','scores':'topic_score'})
df_sentiment['sentiment'] = df_sentiment['sentiment'].str.lower()


In [5]:
df_merged = df_emot.merge(df_sentiment, left_index=True, right_index=True)
df_merged = df_merged.merge(df_topic,left_index=True,right_index=True)
#df_merged = df_merged.rename(columns={'cleaned_text_x': 'cleaned_text'})
df_merged = df_merged.drop(['cleaned_text_x','cleaned_text_y'],axis=1)
df_merged = df_merged.sort_values(by="created_at")
df_merged['created_at']= pd.to_datetime(df_merged['created_at'], format='%Y-%m-%d')
df_merged

,created_at,test_id,impressions,clicks,ctr,emotion,emotion_score,sentiment,sentiment_score,cleaned_text,topic,topic_score
48716,2013-01-24,5143608d220cb800020009e2,6983,105,0.015037,joy,0.991908,negative,0.998309,A Culture That Refuses To Speak About Pregnanc...,Society and Social Issues,0.553050
133752,2013-01-24,51436070220cb8000200074e,3486,122,0.034997,joy,0.986707,positive,0.997439,A Woman Lived Through Something Awful Now Shes...,Society and Social Issues,0.381437
48810,2013-01-24,5143608d220cb800020009e2,7044,77,0.010931,joy,0.990454,negative,0.998423,What It Means To Change The Perception Of Who ...,Society and Social Issues,0.605390
50444,2013-01-24,5143608d220cb800020009e2,6986,109,0.015603,joy,0.994656,negative,0.998072,Watch One Blogger Calmly Explain Why Shes Sick...,"Pop Culture, Media and Entertainment",0.506504
133930,2013-01-24,51436070220cb8000200074e,3364,90,0.026754,joy,0.975622,positive,0.995158,Maybe Shes Born With It Or She Just Bleached T...,"Pop Culture, Media and Entertainment",0.463655
...,...,...,...,...,...,...,...,...,...,...,...,...
25969,2015-04-30,55418d7b313132002c070000,2043,22,0.010768,anger,0.823319,positive,0.706596,When Elizabeth Warren Was 12 Years Old Somethi...,"Relationships, Gender and Family",0.438197
25970,2015-04-30,55418d7b313132002c070000,1953,18,0.009217,anger,0.823319,positive,0.706596,Theres A Reason Elizabeth Warren Thinks The Mi...,Society and Social Issues,0.690894
25962,2015-04-30,55417f91343635002c7e0000,2062,39,0.018914,anger,0.823319,positive,0.706596,Beyoncés Epic Girl Power Anthem That Kinda Sor...,"Pop Culture, Media and Entertainment",0.683434
25963,2015-04-30,55417f91343635002c7e0000,1962,31,0.015800,anger,0.823319,positive,0.706596,Inevitably Someone Will Say But My Juice Clean...,"Authenticity, Lifestyle and Health",0.622102


In [6]:
df_merged['sentiment'] = np.where((df_merged['sentiment_score'] <= 0.9), 'unclassified sentiment', df_merged['sentiment'])
df_merged['emotion'] = np.where((df_merged['emotion_score'] <= 0.75) , 'unclassified emotion', df_merged['emotion'])
df_merged['topic'] = np.where((df_merged['topic_score'] <= 0.4) , 'unclassified topic', df_merged['topic'])


In [7]:
daily_sentiment = df_merged.groupby(['created_at', 'sentiment']).size().unstack(fill_value=0)
daily_emotion = df_merged.groupby(['created_at', 'emotion']).size().unstack(fill_value=0)
daily_topic = df_merged.groupby(['created_at', 'topic']).size().unstack(fill_value=0)

daily_sentiment_proportions = daily_sentiment.div(daily_sentiment.sum(axis=1), axis=0)
daily_emotion_proportions = daily_emotion.div(daily_emotion.sum(axis=1), axis=0)
daily_topic_proportions = daily_topic.div(daily_topic.sum(axis=1), axis=0)

# Define the rolling window size
window_size = 30

# Calculate rolling mean and standard deviation
rolling_sentiment_mean = rolling_sentiment_mean.fillna(method='ffill')  # Forward fill
rolling_sentiment_std = rolling_sentiment_std.fillna(method='ffill')

# You can do the same for emotion and topic data
rolling_emotion_mean = rolling_emotion_mean.fillna(method='ffill')
rolling_emotion_std = rolling_emotion_std.fillna(method='ffill')

rolling_topic_mean = rolling_topic_mean.fillna(method='ffill')
rolling_topic_std = rolling_topic_std.fillna(method='ffill')

NameError: name 'rolling_sentiment_mean' is not defined

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller

def clean_data(data):
    # Replace inf/-inf with NaN and forward/backward fill
    return data.replace([np.inf, -np.inf], np.nan).fillna(method='ffill').fillna(method='bfill')

def plot_column(ax, column_data, std_data, title, font_size):
    # Clean the data
    column_data = clean_data(column_data)
    std_data = clean_data(std_data)

    # Check if data still contains NaNs or infinities
    if not column_data.isna().any() and not np.isinf(column_data).any():
        # Perform Augmented Dickey-Fuller test
        adf_result = adfuller(column_data)
        p_value = adf_result[1]
        adf_label = f'{title} (ADF p-value: {p_value:.2f})'
    else:
        adf_label = f'{title} (ADF test not performed due to NaNs/Infinities)'

    # Ensuring that standard deviation doesn't go below zero
    lower_std = column_data - std_data
    lower_std[lower_std < 0] = 0

    # Plotting the mean and standard deviation
    ax.plot(column_data, label=adf_label)
    ax.fill_between(column_data.index, lower_std, column_data + std_data, alpha=0.2)

def plot_with_rolling_mean(ax, mean_data, std_data, title, font_size):
    for column in mean_data.columns:
        plot_column(ax, mean_data[column], std_data[column], f"{title} - {column}", font_size)

    # Setting the title and labels
    ax.set_title(title)
    ax.set_ylabel('Proportions')

    # Adding the legend
    ax.legend()
# Example use, replace with your actual data
daily_sentiment = df_merged.groupby(['created_at', 'sentiment']).size().unstack(fill_value=0)
daily_emotion = df_merged.groupby(['created_at', 'emotion']).size().unstack(fill_value=0)
daily_topic = df_merged.groupby(['created_at', 'topic']).size().unstack(fill_value=0)

daily_sentiment_proportions = daily_sentiment.div(daily_sentiment.sum(axis=1), axis=0)
daily_emotion_proportions = daily_emotion.div(daily_emotion.sum(axis=1), axis=0)
daily_topic_proportions = daily_topic.div(daily_topic.sum(axis=1), axis=0)

# Define the rolling window size
window_size = 30

# Calculate rolling mean and standard deviation
rolling_sentiment_mean = rolling_sentiment_mean.fillna(method='ffill')  # Forward fill
rolling_sentiment_std = rolling_sentiment_std.fillna(method='ffill')

# You can do the same for emotion and topic data
rolling_emotion_mean = rolling_emotion_mean.fillna(method='ffill')
rolling_emotion_std = rolling_emotion_std.fillna(method='ffill')

rolling_topic_mean = rolling_topic_mean.fillna(method='ffill')
rolling_topic_std = rolling_topic_std.fillna(method='ffill')
import matplotlib.pyplot as plt
import pandas as pd

# ... (Include the definitions of clean_data and plot_with_rolling_mean functions here)

# Assuming you have your dataframes ready
# rolling_sentiment_mean, rolling_sentiment_std, etc.

fig, axs = plt.subplots(3, 1, figsize=(12, 20), gridspec_kw={'height_ratios': [1, 1, 1]})

# Adjust font sizes for better readability
plt.rc('font', size=16)
plt.rc('axes', titlesize=16)
plt.rc('axes', labelsize=16)
plt.rc('legend', fontsize=13)

# Plotting rolling mean for sentiment, emotion, and topic
plot_with_rolling_mean(axs[0], rolling_sentiment_mean, rolling_sentiment_std, 'Sentiment Proportions', 16)
plot_with_rolling_mean(axs[1], rolling_emotion_mean, rolling_emotion_std, 'Emotion Proportions', 16)
plot_with_rolling_mean(axs[2], rolling_topic_mean, rolling_topic_std, 'Topic Proportions', 16)

plt.tight_layout()
plt.savefig('proportions_rolling_means.pdf', format='pdf', bbox_inches='tight', dpi=1200)
plt.show()


In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(12, 16), gridspec_kw={'height_ratios': [1, 1, 1]})

# Adjust font sizes for better readability
plt.rc('font', size=16)
plt.rc('axes', titlesize=16)
plt.rc('axes', labelsize=16)
plt.rc('legend', fontsize=16)

# Histogram for sentiment
df_merged['sentiment'].value_counts(normalize=True).plot(kind='bar', ax=axs[0], color='green')
axs[0].set_title('Sentiment Distribution')
axs[0].set_ylabel('Normalized Count')
axs[0].set_xticklabels(split_labels(df_merged['sentiment'].value_counts().index), rotation=45)

# Histogram for emotion
df_merged['emotion'].value_counts(normalize=True).plot(kind='bar', ax=axs[1], color='blue')
axs[1].set_title('Emotion Distribution')
axs[1].set_ylabel('Normalized Count')
axs[1].set_xticklabels(split_labels(df_merged['emotion'].value_counts().index), rotation=45)

# Histogram for topic
df_merged['topic'].value_counts(normalize=True).plot(kind='bar', ax=axs[2], color='red')
axs[2].set_title('Topic Distribution')
axs[2].set_ylabel('Normalized Count')
axs[2].set_xticklabels(split_labels(df_merged['topic'].value_counts().index), rotation=45)

plt.tight_layout()
plt.savefig('normalized_count_distributions.pdf', format='pdf', bbox_inches='tight', dpi=1200)
plt.show()

# Group aggregated ctr

In [ ]:
triple_stats = df_merged.groupby(['emotion', 'sentiment', 'topic']).agg(mean_ctr=('ctr', 'mean'), counts=('ctr', 'size')).reset_index()
total_counts = triple_stats['counts'].sum()

# Normalize the counts
triple_stats['group_normalized_counts'] = triple_stats['counts'] / total_counts
triple_stats = triple_stats.sort_values(by='mean_ctr',ascending=False)
# Display the results
triple_stats['rank_aggregated_triplet'] = triple_stats['mean_ctr'].rank(method='dense', ascending=False).astype(int)
triple_stats

# Individual articles (not aggregated), top performing, maybe one article popular but not the others

In [ ]:

# Calculate the top 5% cutoff
top_5_percent_cutoff = int(len(df_merged) * 0.05)

# Select the top 5% articles by CTR
top_crt_df = df_merged.sort_values(by='ctr', ascending=False).head(top_5_percent_cutoff)

# Group by 'emotion', 'sentiment', and 'topic', and count the occurrences
triplet_counts = top_crt_df.groupby(['emotion', 'sentiment', 'topic']).size().reset_index(name='counts')

# Calculate the total number of entries in top_crt_df
total_counts =  len(top_crt_df)

# Normalize the counts
triplet_counts['ind_normalized_counts'] = triplet_counts['counts'] / total_counts

# Sort the data for better readability (optional)
triplet_counts = triplet_counts.sort_values(by='ind_normalized_counts', ascending=False)

# Display the normalized triplet counts
triplet_counts.reset_index(drop=True, inplace=True)

# Add a rank column based on 'normalized_counts'
triplet_counts['rank_individual_occurence'] = triplet_counts['ind_normalized_counts'].rank(method='dense', ascending=False).astype(int)
triplet_counts_individual = triplet_counts.copy()
triplet_counts_individual

# Dailly aggregates


In [ ]:

aggregations = {
    'impressions': 'sum',
    'clicks': 'sum',
    'ctr':'mean'
}
non_numeric_columns = ['test_id', 'cleaned_text','emotion','sentiment','topic']
aggregations.update({col: 'first' for col in non_numeric_columns})

# Group by 'created_at' and apply the aggregation
df_daily_agg = df_merged.groupby('created_at').agg(aggregations).reset_index()
df_daily_agg = df_daily_agg.sort_values(by='created_at')

# Ensure 'created_at' is a datetime
df_daily_agg['created_at'] = pd.to_datetime(df_daily_agg['created_at'])

# Create a complete datetime index
complete_date_range = pd.date_range(start=df_daily_agg['created_at'].min(), 
                                    end=df_daily_agg['created_at'].max(), 
                                    freq='D')

# Reindex the DataFrame - making sure that 'created_at' is the index
df_daily_agg_reindexed = df_daily_agg.set_index('created_at').reindex(complete_date_range, method=None).rename_axis('created_at').reset_index()

# Forward fill missing values
df_daily_agg_filled = df_daily_agg_reindexed.ffill()

# Calculate daily CTR
df_daily_agg_filled['daily_ctr'] = df_daily_agg_filled['clicks'] / df_daily_agg_filled['impressions']

# Check final DataFrame
df_daily_agg = df_daily_agg_filled
df_daily_agg

In [ ]:
# Group by 'created_at' and 'triplet' (emotion, sentiment, topic), then sum the clicks and impressions for each group
grouped = df_merged.groupby(['created_at', 'emotion', 'sentiment', 'topic']).agg({'clicks': 'sum', 'impressions': 'sum'}).reset_index()

# Compute the CTR for each group
grouped['daily_group_ctr'] = grouped['clicks'] / grouped['impressions']

# Sort the groups by date and CTR, then select the top triplet for each day
top_triplet_each_day = grouped.sort_values(['created_at', 'daily_group_ctr'], ascending=[True, False]).groupby('created_at').head(1)

# Now you can proceed with counting, normalizing, and ranking these triplets


In [ ]:
# Count how many times each triplet appears
triplet_counts = top_triplet_each_day.groupby(['emotion', 'sentiment', 'topic']).size().reset_index(name='counts')

# Calculate normalized counts
total_counts = triplet_counts['counts'].sum()
triplet_counts['daily_normalized_counts'] = triplet_counts['counts'] / total_counts

# Sort the triplets by normalized counts
triplet_counts_sorted = triplet_counts.sort_values(by='daily_normalized_counts', ascending=False)

# Rank the triplets
triplet_counts_sorted['rank_daily_top_contribution'] = triplet_counts_sorted['daily_normalized_counts'].rank(method='dense', ascending=False).astype(int)

# Display the sorted and ranked triplets
triplet_counts_daily = triplet_counts_sorted
triplet_counts_daily

# Comparing the rankings

In [ ]:
from scipy.stats import spearmanr
# Merge the first two dataframes
merged_df = pd.merge(triple_stats, triplet_counts_individual, on=['emotion', 'sentiment', 'topic'], how='inner')

# Now merge the result with the third dataframe
triple_ranks = pd.merge(merged_df, triplet_counts_daily, on=['emotion', 'sentiment', 'topic'], how='inner')

# Selecting specific columns and sorting
triple_ranks = triple_ranks[['emotion', 'sentiment', 'topic', 'rank_individual_occurence', 'rank_aggregated_triplet', 'rank_daily_top_contribution','daily_normalized_counts']].sort_values(by='rank_individual_occurence')

# Display the DataFrame

# Mean clickbait rank

triple_ranks['mean_clickbait_rank'] = triple_ranks[['rank_individual_occurence', 'rank_aggregated_triplet', 'rank_daily_top_contribution']].mean(axis=1)

# Sort the DataFrame by the new combined rank
triple_ranks_sorted = triple_ranks.sort_values(by='mean_clickbait_rank')

# Select specific columns and sort
# Assuming you want to include new columns like 'rank_peaks', 'normalized_counts_peaks', etc.
selected_columns = [
    'emotion', 'sentiment', 'topic', 'mean_clickbait_rank',
    'rank_individual_occurence',
    'rank_aggregated_triplet',
    'rank_daily_top_contribution', 
]

triple_ranks = triple_ranks[selected_columns].sort_values(by='mean_clickbait_rank')

# Display the DataFrame
# Assuming your DataFrame is named triple_ranks
# Add a new column for the combined rank


# Display the DataFrame
triple_ranks.head(10)


In [ ]:
from scipy.stats import spearmanr
import pandas as pd

# Assuming triple_ranks is your DataFrame
columns = ['rank_individual_occurence', 'rank_aggregated_triplet', 'rank_daily_top_contribution']
correlation_matrix = pd.DataFrame(index=columns, columns=columns)

# Calculate Spearman rank correlation and check for significance
for col1 in columns:
    for col2 in columns:
        corr, p_value = spearmanr(triple_ranks[col1], triple_ranks[col2])
        # Mark with two asterisks if p-value is less than 0.01, one asterisk if less than 0.05
        if p_value < 0.01:
            correlation_matrix.at[col1, col2] = f"{corr:.2f}**"
        elif p_value < 0.05:
            correlation_matrix.at[col1, col2] = f"{corr:.2f}*"
        else:
            correlation_matrix.at[col1, col2] = f"{corr:.2f}"

# Display the correlation matrix
correlation_matrix


In [ ]:
# This ones are hit or miss that are daily performers typically
# But the groups show that on average, they drive consistent traffic


In [ ]:
# Assuming triple_ranks is your DataFrame with 'emotion', 'sentiment', 'topic', and 'mean_clickbait_rank'

# Grouping by emotion
grouped_emotion = triple_ranks.groupby('emotion')['mean_clickbait_rank'].mean().reset_index()
std_emotion = triple_ranks.groupby('emotion')['mean_clickbait_rank'].std().reset_index()

# Grouping by sentiment
grouped_sentiment = triple_ranks.groupby('sentiment')['mean_clickbait_rank'].mean().reset_index()
std_sentiment = triple_ranks.groupby('sentiment')['mean_clickbait_rank'].std().reset_index()

# Grouping by topic
grouped_topic = triple_ranks.groupby('topic')['mean_clickbait_rank'].mean().reset_index()
std_topic = triple_ranks.groupby('topic')['mean_clickbait_rank'].std().reset_index()


In [ ]:
from scipy import stats

# Assuming triple_ranks is your DataFrame
categories = ['emotion', 'sentiment', 'topic']

for category in categories:
    print(f"\nCategory: {category}")
    
    # Get unique group names for the current category
    group_names = triple_ranks[category].unique()

    # Normality test for each group
    print("Shapiro-Wilk Test for Normality:")
    for group in group_names:
        group_data = triple_ranks[triple_ranks[category] == group]['mean_clickbait_rank']
        shapiro_test = stats.shapiro(group_data)
        print(f"{group}: Statistics={shapiro_test.statistic}, p-value={shapiro_test.pvalue}")
        if shapiro_test.pvalue < 0.05:
            print(f"  - The data for {group} in {category} is not normally distributed.")
        else:
            print(f"  - The data for {group} in {category} appears to be normally distributed.")

    # Homogeneity of variances test across groups
    print("\nLevene's Test for Homogeneity of Variances:")
    levene_test = stats.levene(*[triple_ranks[triple_ranks[category] == group]['mean_clickbait_rank'] for group in group_names])
    print(f"Statistics={levene_test.statistic}, p-value={levene_test.pvalue}")
    if levene_test.pvalue < 0.05:
        print(f"  - There is evidence to suggest that variances are not equal across the {category} groups.")
    else:
        print(f"  - Variances appear to be equal across the {category} groups.")


In [ ]:
from scipy import stats

# Assuming triple_ranks is your DataFrame
categories = ['emotion', 'sentiment', 'topic']

for category in categories:
    print(f"\nCategory: {category}")

    # Extracting the groups for the current category
    groups = [group['mean_clickbait_rank'].values for name, group in triple_ranks.groupby(category)]

    # Kruskal-Wallis H Test
    kw_result = stats.kruskal(*groups)
    print(f"Kruskal-Wallis H Test for {category}: Statistics={kw_result.statistic}, p-value={kw_result.pvalue}")

    # Interpret the results
    if kw_result.pvalue < 0.05:
        print("  - There is a statistically significant difference between groups in this category.")
    else:
        print("  - No statistically significant difference was found between groups in this category.")


In [ ]:
kw_result

In [ ]:
import matplotlib.pyplot as plt

# Define color mappings
emotion_colors = {'anger': 'red', 'joy': 'yellow', 'sadness': 'blue', 'fear': 'black', 'surprise': 'orange','love':'pink', 'unclassified_emotion':'gray'}
sentiment_colors = {'positive': 'green', 'negative': 'red', 'unclassified sentiment': 'gray'}
topic_colors = {'Authenticity, Lifestyle and Health': 'green', 
                'Society and Social Issues': 'cyan', 
                'Pop Culture, Media and Entertainment': 'magenta', 
                'Relationships, Gender and Family':'red',
                'unclassified topic': 'gray'}

# Function to get colors for each bar
def get_colors(grouped_data, color_map, column_name):
    colors = [color_map.get(x, 'gray') for x in grouped_data[column_name]]
    return colors

import matplotlib.pyplot as plt

# Assuming triple_ranks is your DataFrame and previous tests have been run
categories = ['emotion', 'sentiment', 'topic']
kw_results = {
    'emotion': {"stat": 36.8467, "p_value": 1.8863e-06},
    'sentiment': {"stat": 7.8058, "p_value": 0.0202},
    'topic': {"stat": 7.1258, "p_value": 0.1294}
}

fig, axs = plt.subplots(3, 1, figsize=(12, 18), gridspec_kw={'height_ratios': [1, 1, 1]})

# Plot for each category
for i, category in enumerate(categories):
    grouped_data = triple_ranks.groupby(category)['mean_clickbait_rank'].mean().reset_index()
    std_data = triple_ranks.groupby(category)['mean_clickbait_rank'].std().reset_index()

    # Define color mappings (update these as per your categories)
    color_mappings = {
        'emotion': emotion_colors,
        'sentiment': sentiment_colors,
        'topic': topic_colors
    }

    bars = axs[i].bar(grouped_data[category], grouped_data['mean_clickbait_rank'], yerr=std_data['mean_clickbait_rank'], color=get_colors(grouped_data, color_mappings[category], category), alpha=0.8)
    axs[i].set_title(f'Mean Clickbait Rank by {category.capitalize()}')
    axs[i].set_ylabel('Mean Clickbait Rank')
    axs[i].set_xticks(range(len(grouped_data[category])))
    axs[i].set_xticklabels(grouped_data[category], rotation=30)

    # Preparing Kruskal-Wallis H Test results for the legend
    kw_result = kw_results[category]
    significant = kw_result["p_value"] < 0.05
    symbol = "✓" if significant else "X"
    stat_text = f'Kruskal-Wallis H: Stat={kw_result["stat"]:.2f}, p={kw_result["p_value"]:.2e}\nSignificance: {symbol}'
    
    # Create a patch for the legend
    patch = mpatches.Patch(color='white', label=stat_text)
    
    # Adding the test result to the legend
    axs[i].legend(handles=[patch], loc='upper right', fontsize=16, frameon=False)

plt.tight_layout()
plt.savefig('triplets_grouped.pdf', format='pdf', bbox_inches='tight', dpi=1200)

# Null model for ranks

In [ ]:

def calculate_triple_stats(df):
    triple_stats = df.groupby(['emotion', 'sentiment', 'topic']).agg(mean_ctr=('ctr', 'mean'), counts=('ctr', 'size')).reset_index()
    total_counts = triple_stats['counts'].sum()

    # Normalize the counts
    triple_stats['group_normalized_counts'] = triple_stats['counts'] / total_counts
    triple_stats = triple_stats.sort_values(by='mean_ctr', ascending=False)

    # Rank the results
    triple_stats['rank_aggregated_triplet'] = triple_stats['mean_ctr'].rank(method='dense', ascending=False).astype(int)

    return triple_stats

In [ ]:
def calculate_triplet_counts_individual(df):
    # Calculate the top 5% cutoff
    top_5_percent_cutoff = int(len(df) * 0.05)

    # Select the top 5% articles by CTR
    top_crt_df = df.sort_values(by='ctr', ascending=False).head(top_5_percent_cutoff)

    # Group by 'emotion', 'sentiment', and 'topic', and count the occurrences
    triplet_counts = top_crt_df.groupby(['emotion', 'sentiment', 'topic']).size().reset_index(name='counts')

    # Calculate the total number of entries in top_crt_df
    total_counts = len(top_crt_df)

    # Normalize the counts
    triplet_counts['ind_normalized_counts'] = triplet_counts['counts'] / total_counts

    # Sort the data for better readability (optional)
    triplet_counts = triplet_counts.sort_values(by='ind_normalized_counts', ascending=False)

    # Add a rank column based on 'normalized_counts'
    triplet_counts['rank_individual_occurence'] = triplet_counts['ind_normalized_counts'].rank(method='dense', ascending=False).astype(int)

    return triplet_counts




In [ ]:
def calculate_triplet_counts_daily(df):
    aggregations = {
        'impressions': 'sum',
        'clicks': 'sum',
        'ctr': 'mean'
    }
    non_numeric_columns = ['test_id', 'cleaned_text', 'emotion', 'sentiment', 'topic']
    aggregations.update({col: 'first' for col in non_numeric_columns})

    # Group by 'created_at' and apply the aggregation
    df_daily_agg = df.groupby('created_at').agg(aggregations).reset_index()
    df_daily_agg = df_daily_agg.sort_values(by='created_at')

    # Ensure 'created_at' is a datetime
    df_daily_agg['created_at'] = pd.to_datetime(df_daily_agg['created_at'])

    # Create a complete datetime index
    complete_date_range = pd.date_range(start=df_daily_agg['created_at'].min(), 
                                        end=df_daily_agg['created_at'].max(), 
                                        freq='D')

    # Reindex the DataFrame - making sure that 'created_at' is the index
    df_daily_agg_reindexed = df_daily_agg.set_index('created_at').reindex(complete_date_range, method=None).rename_axis('created_at').reset_index()

    # Forward fill missing values
    df_daily_agg_filled = df_daily_agg_reindexed.ffill()

    # Calculate daily CTR
    df_daily_agg_filled['daily_ctr'] = df_daily_agg_filled['clicks'] / df_daily_agg_filled['impressions']

    # Group by 'created_at' and 'triplet' (emotion, sentiment, topic), then sum the clicks and impressions for each group
    grouped = df.groupby(['created_at', 'emotion', 'sentiment', 'topic']).agg({'clicks': 'sum', 'impressions': 'sum'}).reset_index()

    # Compute the CTR for each group
    grouped['daily_group_ctr'] = grouped['clicks'] / grouped['impressions']

    # Sort the groups by date and CTR, then select the top triplet for each day
    top_triplet_each_day = grouped.sort_values(['created_at', 'daily_group_ctr'], ascending=[True, False]).groupby('created_at').head(1)

    # Count how many times each triplet appears
    triplet_counts = top_triplet_each_day.groupby(['emotion', 'sentiment', 'topic']).size().reset_index(name='counts')

    # Calculate normalized counts
    total_counts = triplet_counts['counts'].sum()
    triplet_counts['daily_normalized_counts'] = triplet_counts['counts'] / total_counts

    # Sort the triplets by normalized counts
    triplet_counts_sorted = triplet_counts.sort_values(by='daily_normalized_counts', ascending=False)

    # Rank the triplets
    triplet_counts_sorted['rank_daily_top_contribution'] = triplet_counts_sorted['daily_normalized_counts'].rank(method='dense', ascending=False).astype(int)

    return triplet_counts_sorted

In [ ]:

def calculate_final_triple_ranks(df):
    # Call the previously defined functions to get the required dataframes
    triple_stats = calculate_triple_stats(df)
    triplet_counts_individual = calculate_triplet_counts_individual(df)
    triplet_counts_daily = calculate_triplet_counts_daily(df)

    # Determine the set of all unique triplets across all dataframes
    all_triplets = set(triple_stats[['emotion', 'sentiment', 'topic']].itertuples(index=False)) | \
                   set(triplet_counts_individual[['emotion', 'sentiment', 'topic']].itertuples(index=False)) | \
                   set(triplet_counts_daily[['emotion', 'sentiment', 'topic']].itertuples(index=False))

    # Create a DataFrame of all unique triplets
    all_triplets_df = pd.DataFrame(all_triplets, columns=['emotion', 'sentiment', 'topic'])

    # Merge the dataframes with all_triplets_df to ensure all triplets are represented
    merged_df = pd.merge(all_triplets_df, triple_stats, on=['emotion', 'sentiment', 'topic'], how='left')
    merged_df = pd.merge(merged_df, triplet_counts_individual, on=['emotion', 'sentiment', 'topic'], how='left')
    merged_df = pd.merge(merged_df, triplet_counts_daily, on=['emotion', 'sentiment', 'topic'], how='left')

    # Assign last ranks for missing values in each ranking column
    max_rank_individual = merged_df['rank_individual_occurence'].max() + 1
    max_rank_aggregated = merged_df['rank_aggregated_triplet'].max() + 1
    max_rank_daily = merged_df['rank_daily_top_contribution'].max() + 1

    merged_df['rank_individual_occurence'].fillna(max_rank_individual, inplace=True)
    merged_df['rank_aggregated_triplet'].fillna(max_rank_aggregated, inplace=True)
    merged_df['rank_daily_top_contribution'].fillna(max_rank_daily, inplace=True)

    # Calculate the mean clickbait rank
    merged_df['mean_clickbait_rank'] = merged_df[['rank_individual_occurence', 'rank_aggregated_triplet', 'rank_daily_top_contribution']].mean(axis=1)

    # Sort the DataFrame by the new combined rank
    triple_ranks_sorted = merged_df.sort_values(by='mean_clickbait_rank')

    return triple_ranks_sorted


## Significance testing

In [ ]:
import pandas as pd
from scipy import stats
import numpy as np

def calculate_z_test(original_mean, shuffled_means, std_dev, n):
    z_score = (original_mean - np.mean(shuffled_means)) / (std_dev / np.sqrt(n))
    p_value = stats.norm.sf(abs(z_score)) * 2
    return p_value

def calculate_mannwhitney_u_test(original_data, shuffled_data):
    u_statistic, p_value = stats.mannwhitneyu(original_data, shuffled_data, alternative='two-sided')
    return p_value

# Assuming df_merged is your original dataframe and calculate_final_triple_ranks is defined
original_ranks = calculate_final_triple_ranks(df_merged)

# Prepare to store results
results = []

# Bonferroni correction
alpha = 0.05 / 2 #Two tests per triplet
trials = 1000

# Perform shuffling for each trial
triplet_shuffled_means, ctr_shuffled_means = {}, {}

# Perform shuffling for each trial
for i in range(trials):
    print(i, trials)

    # Shuffle for triplet and ctr
    df_shuffle_triplet = df_merged.copy()
    df_shuffle_triplet['sentiment'] = df_shuffle_triplet['sentiment'].sample(frac=1).reset_index(drop=True)
    df_shuffle_triplet['emotion'] = df_shuffle_triplet['emotion'].sample(frac=1).reset_index(drop=True)
    df_shuffle_triplet['topic'] = df_shuffle_triplet['topic'].sample(frac=1).reset_index(drop=True)

    df_shuffle_ctr = df_merged.copy()
    df_shuffle_ctr['ctr'] = df_shuffle_ctr['ctr'].sample(frac=1).reset_index(drop=True)

    shuffled_rank_triplet = calculate_final_triple_ranks(df_shuffle_triplet)
    shuffled_rank_ctr = calculate_final_triple_ranks(df_shuffle_ctr)

    # Iterate over each triplet in the original ranks
    for index, row in original_ranks.iterrows():
        emotion, sentiment, topic = row['emotion'], row['sentiment'], row['topic']
        triplet_key = (emotion, sentiment, topic)

        shuffled_mean_triplet = shuffled_rank_triplet.loc[(shuffled_rank_triplet['emotion'] == emotion) & 
                                                          (shuffled_rank_triplet['sentiment'] == sentiment) & 
                                                          (shuffled_rank_triplet['topic'] == topic), 'mean_clickbait_rank'].mean()
        shuffled_mean_ctr = shuffled_rank_ctr.loc[(shuffled_rank_ctr['emotion'] == emotion) & 
                                                  (shuffled_rank_ctr['sentiment'] == sentiment) & 
                                                  (shuffled_rank_ctr['topic'] == topic), 'mean_clickbait_rank'].mean()

        triplet_shuffled_means.setdefault(triplet_key, []).append(shuffled_mean_triplet)
        ctr_shuffled_means.setdefault(triplet_key, []).append(shuffled_mean_ctr)

# Perform Shapiro-Wilk test and choose appropriate test (Z-Test or Mann-Whitney)
for triplet_key in original_ranks[['emotion','sentiment','topic']].values:
    triplet_key = tuple(triplet_key)  # Convert numpy.ndarray to tuple
    emotion, sentiment, topic = triplet_key
    original_mean = original_ranks.loc[(original_ranks['emotion'] == emotion) & 
                                       (original_ranks['sentiment'] == sentiment) & 
                                       (original_ranks['topic'] == topic), 'mean_clickbait_rank'].mean()

    for shuffled_means, shuffle_type in [(triplet_shuffled_means[triplet_key], 'Triplet Shuffle'), (ctr_shuffled_means[triplet_key], 'CTR Shuffle')]:
        if len(shuffled_means) >= 3:
            shapiro_test = stats.shapiro(shuffled_means)
            p_value_shapiro = shapiro_test.pvalue

            if p_value_shapiro >= alpha:
                p_value_test = calculate_z_test(original_mean, shuffled_means, np.std(shuffled_means), len(shuffled_means))
                test_type = 'Z-Test'
            else:
                p_value_test = calculate_mannwhitney_u_test([original_mean], shuffled_means)
                test_type = 'Mann-Whitney U'

            results.append({
                'emotion': emotion,
                'sentiment': sentiment,
                'topic': topic,
                'shuffle_type': shuffle_type,
                'shapiro_p_value': p_value_shapiro,
                'test_type': test_type,
                'p_value_test': p_value_test
            })
        else:
            pass
# Convert results to DataFrame
results_df = pd.DataFrame(results)
results_df['stat_significant'] = results_df.apply(lambda x: 'Statistically Significant' if x['p_value_test']<alpha else 'Not Statistically Significant',axis=1)
# Print and save the results
print(results_df)


In [ ]:
results_df['stat_significant'] = results_df.apply(lambda x: 'Statistically Significant' if x['p_value_test']<alpha else 'Not Statistically Significant',axis=1)

In [ ]:

# Recreating the plots with logarithmic scale on the x-axis

# Filter out rows with p-values of zero or NaN
filtered_results_df = results_df[(results_df['p_value_test'] > 0) & (results_df['p_value_test'].notna())]

# Recalculate the filtered DataFrames and counts
triplet_shuffle = filtered_results_df[filtered_results_df['shuffle_type'] == 'Triplet Shuffle']
ctr_shuffle = filtered_results_df[filtered_results_df['shuffle_type'] == 'CTR Shuffle']
triplet_counts = triplet_shuffle['stat_significant'].value_counts()
ctr_counts = ctr_shuffle['stat_significant'].value_counts()

# Define alpha threshold for statistical significance
alpha_threshold = alpha

min_p_value = max(1e-10, min(filtered_results_df['p_value_test']))
max_p_value = max(filtered_results_df['p_value_test'])
bin_edges = np.logspace(np.log10(min_p_value), np.log10(max_p_value), 20)

# Create the plots with adjusted bin ranges and log scale for x-axis
fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=True)


# Plot histograms with color based on statistical significance
sns.histplot(triplet_shuffle, x='p_value_test', hue='stat_significant', 
             palette={'Statistically Significant': 'green', 'Not Statistically Significant': 'red'}, 
             ax=axes[0], kde=False, bins=20, log_scale=True)
sns.histplot(ctr_shuffle, x='p_value_test', hue='stat_significant', 
             palette={'Statistically Significant': 'green', 'Not Statistically Significant': 'red'}, 
             ax=axes[1], kde=False, bins=20, log_scale=True)

# Add alpha threshold lines
axes[0].axvline(alpha_threshold, color='blue', linestyle='--')
axes[1].axvline(alpha_threshold, color='blue', linestyle='--')

# Set titles and labels
axes[0].set_title('Triplet Shuffle P-Values')
axes[0].set_xlabel('P-Value')
axes[0].set_ylabel('Count')

axes[1].set_title('CTR Shuffle P-Values')
axes[1].set_xlabel('P-Value')

# Create custom legend entries
sig_patch = mpatches.Patch(color='none', label=f'Significant: {triplet_counts.get("Statistically Significant", 0)}')
not_sig_patch = mpatches.Patch(color='none', label=f'Not Significant: {triplet_counts.get("Not Statistically Significant", 0)}')
alpha_patch = mpatches.Patch(color='blue', label='Alpha Threshold')

# Add legends with custom entries
axes[0].legend(handles=[alpha_patch, sig_patch, not_sig_patch], title='Triplet Shuffle')
axes[1].legend(handles=[alpha_patch, sig_patch, not_sig_patch], title='CTR Shuffle')

# Remove grid
axes[0].grid(False)
axes[1].grid(False)

plt.tight_layout()
plt.savefig('null_model_results.pdf', format='pdf', bbox_inches='tight', dpi=1200)
plt.show()

